<a href="https://colab.research.google.com/github/Yang-J1225/dataset/blob/main/vrt_demo_on_video_restoration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# VRT Online Demo on Video Restoration

[![arXiv](https://img.shields.io/badge/arXiv-Paper-<COLOR>.svg)](https://arxiv.org/abs/2201.12288)
[![GitHub Stars](https://img.shields.io/github/stars/JingyunLiang/VRT?style=social)](https://github.com/JingyunLiang/VRT)
[![download](https://img.shields.io/github/downloads/JingyunLiang/VRT/total.svg)](https://github.com/JingyunLiang/VRT/releases)
![visitors](https://visitor-badge.glitch.me/badge?page_id=jingyunliang/VRT)

This is a **VRT online demo on Video Restoration** of our paper [''VRT: A Video Restoration Transformer''](https://arxiv.org/abs/2201.12288).

VRT ahcieves state-of-the-art performance **(up to 2.16dB)** in
- video SR (REDS, Vimeo90K, Vid4 and UDM10)
- video deblurring (GoPro, DVD and REDS)
- video denoising (DAVIS and Set8)

<img src="https://raw.githubusercontent.com/JingyunLiang/VRT/main/assets/vsr_visual.jpeg" width="100%">
<img src="https://raw.githubusercontent.com/JingyunLiang/VRT/main/assets/vdb_visual.jpeg" width="100%">



# 1. Preparations
Before start, make sure that you choose
* Runtime Type = Python 3
* Hardware Accelerator = GPU
* Broswer != Firefox (cannot upload images in step 2)

in the **Runtime** menu -> **Change runtime type**

Then, we clone the repository, set up the envrironment, and download the pre-trained model.

In [1]:
!rm -r VRT
# Clone VRT
!git clone https://github.com/JingyunLiang/VRT.git
%cd VRT
!pip install -r requirements.txt

rm: cannot remove 'VRT': No such file or directory
Cloning into 'VRT'...
remote: Enumerating objects: 99, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 99 (delta 26), reused 29 (delta 9), pack-reused 46
Receiving objects: 100% (99/99), 13.07 MiB | 15.49 MiB/s, done.
Resolving deltas: 100% (28/28), done.
/content/VRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 4.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidi

# 2. Upload Images

Upload the images to be processed by VRT

In [2]:
import os
import glob
from google.colab import files
import shutil
print(' Note1: You can upload video/ video frmaes as a batch. \n Note2: If you just to want to replicate the results in the paper, you can skip this step (we have enabled automatic download in the code).\n Note3: You may need Chrome to enable file uploading!\n Note4: Our model is trained on simple synthetic videos (e.g., bicubic). It may not perform well on real-world videos.\n')
input_folder = None

# upload images
!rm -r testsets/*
!rm -r results/*
os.mkdir('testsets/uploaded')
os.mkdir('testsets/uploaded/000')
uploaded = files.upload()
for filename in uploaded.keys():
  input_folder = 'testsets/uploaded/000'
  dst_path = os.path.join(input_folder, filename)
  print(f'move {filename} to {dst_path}')
  shutil.move(filename, dst_path)

  if not filename.split('.')[-1].lower() in ['png','jpeg','jpg','tiff','bmp','tif',]: # image or video
    print(f'extracting the video as frames to {input_folder}')
    os.system(f'ffmpeg -i testsets/uploaded/{filename} -qscale:v 1 -qmin 1 -qmax 1 -vsync 0  {input_folder}/frame%08d.png')
!ls testsets/uploaded

 Note1: You can upload video/ video frmaes as a batch. 
 Note2: If you just to want to replicate the results in the paper, you can skip this step (we have enabled automatic download in the code).
 Note3: You may need Chrome to enable file uploading!
 Note4: Our model is trained on simple synthetic videos (e.g., bicubic). It may not perform well on real-world videos.

rm: cannot remove 'results/*': No such file or directory


000


# 3. Inference


In [11]:
# Testing arguments
# --task: 001_VRT_videosr_bi_REDS_6frames/
#         002_VRT_videosr_bi_REDS_16frames/
#         003_VRT_videosr_bi_Vimeo_7frames/
#         004_VRT_videosr_bd_Vimeo_7frames/
#         005_VRT_videodeblurring_DVD/
#         006_VRT_videodeblurring_GoPro/
#         007_VRT_videodeblurring_REDS/
#         008_VRT_videodenoising_DAVIS
# --folder_lq: path to low-quality videos. If you have uploaded your own video, it will use yours.
#              Otherwise, it will automatically download benchmark datasets and test on them.
# --folder_gt: path to ground-truth videos. No use for your own video. Only support benchmark datasets.
# --tile: temporal, height and width testing sizes. For example, [40,128,128] means testing the video as 40x128x128 video clips.
#         Reduce it if out-of-memory (at the expense of lower performance).
# --tile_overlap: tile overlapping size. No need to modify it.



##                                       ### test your own video (no ground-truth) ###
!python main_test_vrt.py --task spmc --folder_lq testsets/media/amax/chenchao/spmc/BIx4/ --tile 20 128 128 --tile_overlap 2 20 20



##                                                ### Benchmak testing  ###
## Note: Due to Colab GPU memory limitation, we use smaller --tile here, which may lead to different performance from the paper.
## Please refer to https://github.com/JingyunLiang/VRT#quick-testing for official arguments.)

## 001, video sr trained on REDS (6 frames), tested on REDS4
# !python main_test_vrt.py --task 001_VRT_videosr_bi_REDS_6frames --folder_lq testsets/REDS4/sharp_bicubic --folder_gt testsets/REDS4/GT --tile 40 64 64 --tile_overlap 2 20 20

## 002, video sr trained on REDS (16 frames), tested on REDS4
# !python main_test_vrt.py --task 002_VRT_videosr_bi_REDS_16frames --folder_lq testsets/REDS4/sharp_bicubic --folder_gt testsets/REDS4/GT --tile 40 64 64 --tile_overlap 2 20 20

## 003, video sr trained on Vimeo (bicubic), tested on Vid4 and Vimeo
# !python main_test_vrt.py --task 003_VRT_videosr_bi_Vimeo_7frames --folder_lq testsets/Vid4/BIx4 --folder_gt testsets/Vid4/GT --tile 32 64 64 --tile_overlap 2 20 20
# !python main_test_vrt.py --task 003_VRT_videosr_bi_Vimeo_7frames --folder_lq testsets/vimeo90k/vimeo_septuplet_matlabLRx4/sequences --folder_gt testsets/vimeo90k/vimeo_septuplet/sequences --tile 8 0 0 --tile_overlap 0 20 20

## 004, video sr trained on Vimeo (blur-downsampling), tested on Vid4, UDM10 and Vimeo
# !python main_test_vrt.py --task 004_VRT_videosr_bd_Vimeo_7frames --folder_lq testsets/Vid4/BDx4 --folder_gt testsets/Vid4/GT --tile 32 64 64 --tile_overlap 2 20 20
# !python main_test_vrt.py --task 004_VRT_videosr_bd_Vimeo_7frames --folder_lq testsets/UDM10/BDx4 --folder_gt testsets/UDM10/GT --tile 32 64 64 --tile_overlap 2 20 20
# !python main_test_vrt.py --task 004_VRT_videosr_bd_Vimeo_7frames --folder_lq testsets/vimeo90k/vimeo_septuplet_BDLRx4/sequences --folder_gt testsets/vimeo90k/vimeo_septuplet/sequences --tile 8 0 0 --tile_overlap 0 20 20

## 005, video deblurring trained and tested on DVD
# !python main_test_vrt.py --task 005_VRT_videodeblurring_DVD --folder_lq testsets/DVD10/test_GT_blurred --folder_gt testsets/DVD10/test_GT --tile 12 128 128 --tile_overlap 2 20 20

## 006, video deblurring trained and tested on GoPro
# !python main_test_vrt.py --task 006_VRT_videodeblurring_GoPro --folder_lq testsets/GoPro11/test_GT_blurred --folder_gt testsets/GoPro11/test_GT --tile 12 128 128 --tile_overlap 2 20 20

## 007, video deblurring trained on REDS, tested on REDS4
# !python main_test_vrt.py --task 007_VRT_videodeblurring_REDS --folder_lq testsets/REDS4/blur --folder_gt testsets/REDS4/GT --tile 12 128 128 --tile_overlap 2 20 20

## 008, video denoising trained on DAVIS (noise level 0-50) and tested on Set8 and DAVIS
# !python main_test_vrt.py --task 008_VRT_videodenoising_DAVIS --sigma 10 --folder_lq testsets/Set8 --folder_gt testsets/Set8 --tile 12 128 128 --tile_overlap 2 20 20
# !python main_test_vrt.py --task 008_VRT_videodenoising_DAVIS --sigma 10  --folder_lq testsets/DAVIS-test --folder_gt testsets/DAVIS-test --tile 12 128 128 --tile_overlap 2 20 20


/usr/local/lib/python3.10/dist-packages/torch/functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3587.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
loading model from ./model_zoo/vrt//content/VRT/model_zoo/vrt/001_VRT_videosr_bi_REDS_6frames.pth
downloading testing dataset spmc
media/amax/chenchao/spmc/BIx4/
media/amax/chenchao/spmc/BIx4/philips_hkc05_001/
media/amax/chenchao/spmc/BIx4/philips_hkc05_001/0017.png
media/amax/chenchao/spmc/BIx4/philips_hkc05_001/0008.png
media/amax/chenchao/spmc/BIx4/philips_hkc05_001/0024.png
media/amax/chenchao/spmc/BIx4/philips_hkc05_001/0012.png
media/amax/chenchao/spmc/BIx4/philips_hkc05_001/0029.png
media/amax/chenchao/spmc/BIx4/philips_hkc05_001/0022.png
media/amax/chenchao/spmc/BIx4/philips_hkc05_001/0014.png
media/amax/chenchao/spmc/BIx4/philips_hkc05_001/0005.png
media/amax/chenchao/s

# 5. Download Results


In [14]:
# Download the results
#!tar -czf VRT_result.zip results/*
files.download('VRT_result.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

MessageError: Error: credential propagation was unsuccessful

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!cp -r /content/VRT/VRT_result.zip /content/drive/MyDrive/